# **Практическое задание**

1. Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо
нейронной сети, работающей airline-passengers (она прилагается вместе с датасетом к
уроку в виде отдельного скрипта) так, чтобы улучшить её точность. Приложите анализ
2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом,
чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший
текст из получившихся и опишите предпринятые для его получения действия. Можно
использовать текст другого произведения
3. *Попробуйте реализовать нейронную сеть архитектуры LSTM на numpy
4. *Предложите свои варианты решения проблемы исчезающего градиента в RNN

In [204]:
import numpy as np
import tensorflow as tf
import random as rn
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [205]:
tf.test.is_built_with_cuda()

True

In [206]:
tf.config.list_physical_devices('GPU')

[]

In [207]:
# Вспомогательные функции

# Фиксирование случайных значений
def randomstate(seed=42):
  np.random.seed(seed)
  tf.random.set_seed(seed)
  rn.seed(seed)

# **IMDB отзывы**

In [208]:
max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 150
batch_size = 100 # увеличьте значение для ускорения обучения

In [209]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Загрузка данных...


In [210]:
# x_train[0]

In [211]:
# словарь сопоставления слово-индекс
word_index = imdb.get_word_index()

# словарь сопоставления индекс-слово
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

# декодирование текста первого элемента датасета
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

In [212]:
# decoded_sequence

In [213]:
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

25000 тренировочные последовательности
25000 тестовые последовательности


In [214]:
len(x_train[0])

218

In [215]:
# обрезаем текст до maxlen
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 150)
x_test shape: (25000, 150)


In [216]:
max_features

20000

In [217]:
# создаем 20000 уникальных векторов для каждого слова размером 128
layer = Embedding(max_features, 128)

In [218]:
layer(x_train[0]).shape

TensorShape([150, 128])

In [219]:
print('Построение модели...')
randomstate()
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(70, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Построение модели...


In [220]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # при увеличении наблюдается переобучение
          validation_data=(x_test, y_test))

# начальная точность на трейне
# 500/500 [==============================] - 87s 171ms/step - loss: 0.4324 - accuracy: 0.7956 - val_loss: 0.3570 - val_accuracy: 0.8422

Процесс обучения...
250/250 [==============================] - 142s 557ms/step - loss: 0.4011 - accuracy: 0.8046 - val_loss: 0.3089 - val_accuracy: 0.8686


In [221]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

# начальная точность на тесте
# 500/500 [==============================] - 13s 25ms/step - loss: 0.3570 - accuracy: 0.8422

250/250 [==============================] - 20s 80ms/step - loss: 0.3089 - accuracy: 0.8686


In [222]:
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Результат при тестировании: 0.3089107275009155
Тестовая точность: 0.8685600161552429


### **Вывод:**

**Accuracy начальной модели:**
  - на трейне <font color='red'>**0.7956**</font>
  - на тесте <font color='red'>**0.8422**</font>
---
 1. Увеличим количество слов из датасета: max_features = 30000.
  
  **Accuracy:**
  - на трейне **0.7922**
  - на тесте **0.8383**

 Точность снизилась.
---
 2. Уменьшим количество слов из датасета: max_features = 15000.
  
  **Accuracy:**
  - на трейне **0.7917**
  - на тесте **0.8411**
  
  Точность снизилась.
---
 3. Увеличим количество слов до отсечения: maxlen = 150.
  
  **Accuracy:**
  - на трейне **0.8034**
  - на тесте **0.8610**
  
  Точность увеличилась.
---
 4. Увеличим размер батча: batch_size = 50.
  
  **Accuracy:**
  - на трейне **0.7951**
  - на тесте **0.8678**
  
  Точность на трейне снизилась, на тесте увеличилась.

---
 5. Увеличим размер внутреннего состояния: LSTM(200).
  
  **Accuracy:**
  - на трейне **0.7904**
  - на тесте **0.8450**
  
  Точность на трейне снизилась, на тесте увеличилась.
---
 6. Уменьшим дропауты: LSTM(dropout=0.1, recurrent_dropout=0.1).
  
  **Accuracy:**
  - на трейне **0.7933**
  - на тесте **0.8368**
  
  Точность снизилась.
---
 7. Увеличим дропауты: LSTM(dropout=0.3, recurrent_dropout=0.3).
  
  **Accuracy:**
  - на трейне **0.7889**
  - на тесте **0.8381**
  
  Точность снизилась.
---
 8. Добавим еще полносвязный слой со следующими параметрами:

 - model.add(Dense(256, activation='tanh'))
  
  **Accuracy:**
  - на трейне **0.7972**
  - на тесте **0.8402**
  
  Точность на трейне увеличилась, на тесте снизилась.


**Accuracy финальной модели:**
  - на трейне <font color='lightseagreen'>**0.8046**</font>
  - на тесте <font color='lightseagreen'>**0.8686**</font>

Некоторые наблюдения:
 - Точность повышается при увеличении таких параметров как: maxlen, batch_size.
 - Модель довольно быстро переобучается, одной эпохи достаточно.
 - Регулирование таких параметров как: max_features, dropout, - не привело к увеличению точности.
 - Также были испробованы разные функции активации, добавление полносвязных слоев, изменение количества нейронов, изменение размерности веторов и др.
 - Итоговая модель выглядит следующим образом:
  - model = Sequential()
  - model = Sequential()
  - model.add(Embedding(max_features, 128))
  - model.add(LSTM(70, dropout=0.2, recurrent_dropout=0.2))
  - model.add(Dense(128, activation='tanh'))
  - model.add(Dense(1, activation='sigmoid'))
  - maxlen = 150
  - batch_size = 100

# **Генерация текста на основе книжки «Мы» Евгения Замятина**

In [223]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [224]:
# построчное чтение из примера с текстом 
with open("/content/drive/MyDrive/Colab Notebooks/Нейронные сети/zamyatin_my.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("cp1251", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
text = text.replace('\r', ' ').replace('             ', ' ').replace('         ', ' ')
text = text[:50000]
chars = set(text)
nb_chars = len(chars)

In [225]:
len(chars)

99

In [226]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [227]:
# index2char

In [228]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [229]:
len(input_chars)

49990

In [230]:
input_chars[0], label_chars[0]

('Евгений За', 'м')

In [231]:
input_chars[1], label_chars[1]

('вгений Зам', 'я')

In [232]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [233]:
X.shape

(49990, 10, 99)

In [234]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 80, 200
NUM_ITERATIONS = 15 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [235]:
'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

randomstate()
model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)

model.add(
    GRU(  
        HIDDEN_SIZE,
        return_sequences=False,
        unroll=True,
        dropout=0.2, 
        recurrent_dropout=0.2
    )
)

model.add(Dense(800, activation='tanh'))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [236]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 0
625/625 [==============================] - 89s 127ms/step - loss: 2.7732
Генерация из посева: понимать, 
понимать, – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – ==================================================
Итерация #: 1
625/625 [==============================] - 79s 127ms/step - loss: 2.4947
Генерация из посева: и Благодет
и Благодеть серенное сторать – я на сторать – я на сторать – я на сторать – я на сторать – я на сторать – я на==================================================
Итерация #: 2
625/625 [==============================] - 81s 130ms/step - loss: 2.3903
Генерация из посева: , да, одна
, да, одна сторать серенное странные сказальное сторать серенное странные сказальное сторать серенное странные==================================================
Итерация #: 3
625/625 [==============================] - 80s 127ms/step - loss: 2.3002
Генерация из посева: я: я решил
я: я решил себе сторали – сторали – стора

Были испробованы изменения следующих параметров: batch_size, hidden_size, оптимизатор ('rmsprop' → 'adam'), добавление полносвязных слоев с количеством нейронов 100/500/800, добавление дополнительного слоя GRU, переменной SEQLEN, функции активации последнего слоя ('softmax' → 'sigmoid'), функции активации полносвязного слоя ('tanh' → 'relu'), dropout; комбинации этих параметров.

Некоторые наблюдения:

 - Точность повышается при увеличении таких параметров как: hidden_size, количество нейронов в полносвязных слоях, количество итераций.
 - Точность повышается при уменьшении таких параметров как: batch_size, dropout.
 - На большой выборке модель учится очень долго.
 - Для лучшего обучения пришлось немного обработать текст, избавиться от лишних пробелов и знаков табуляции.
 - Для распознования кириллицы нужна верная кодировка 'cp1251'.

# **Варианты решения проблемы исчезающего градиента в RNN:**
 - перепроектировать модель
 - использовать обрезку градиента
 - использовать регуляризацию весов
 - правильно выбрать функцию активации
 - использовать остаточные нейронные сети (ResNets)
